<a href="https://colab.research.google.com/github/asif2042/AI_Models/blob/main/Next_Word_Prediction_LSTM/Next_word_predictor_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

cp: cannot stat 'kaggle.json': No such file or directory


In [2]:

!kaggle datasets download ronikdedhia/next-word-prediction

Dataset URL: https://www.kaggle.com/datasets/ronikdedhia/next-word-prediction
License(s): unknown
100% 228k/228k [00:00<00:00, 783kB/s]
100% 228k/228k [00:00<00:00, 783kB/s]


In [3]:
import zipfile

zip = zipfile.ZipFile('next-word-prediction.zip', 'r')
zip.extractall('/content/dataset')
zip.close()

In [4]:
def read_text(path):
  with open(path,'r', encoding = 'utf-8') as file:
    text = file.read()
  return text

text = read_text('/content/dataset/1661-0.txt')
print(text[:1000])


﻿
Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.net


Title: The Adventures of Sherlock Holmes

Author: Arthur Conan Doyle

Release Date: November 29, 2002 [EBook #1661]
Last Updated: May 20, 2019

Language: English

Character set encoding: UTF-8

*** START OF THIS PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES ***



Produced by an anonymous Project Gutenberg volunteer and Jose Menendez



cover



The Adventures of Sherlock Holmes



by Arthur Conan Doyle



Contents


   I.     A Scandal in Bohemia
   II.    The Red-Headed League
   III.   A Case of Identity
   IV.    The Boscombe Valley Mystery
   V.     The Five Orange Pips
   VI.    The Man with the Twisted Lip
   VII.   The Adventure of the Blue 

In [5]:
len(text)

581888

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [7]:
tokenizer  = Tokenizer()

In [8]:
tokenizer.fit_on_texts([text])

In [9]:
tot_unique_word = len(tokenizer.word_index)
tot_unique_word

8931

In [10]:
input_sequences = []
for sentence in text.split('\n'):
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1, len(tokenized_sentence)):
   input_sequences.append(tokenized_sentence[:i +1])


In [11]:
input_sequences

[[145, 4790],
 [145, 4790, 1],
 [145, 4790, 1, 1020],
 [145, 4790, 1, 1020, 4],
 [145, 4790, 1, 1020, 4, 128],
 [145, 4790, 1, 1020, 4, 128, 34],
 [145, 4790, 1, 1020, 4, 128, 34, 45],
 [145, 4790, 1, 1020, 4, 128, 34, 45, 611],
 [145, 4790, 1, 1020, 4, 128, 34, 45, 611, 2235],
 [145, 4790, 1, 1020, 4, 128, 34, 45, 611, 2235, 2236],
 [30, 1021],
 [30, 1021, 15],
 [30, 1021, 15, 23],
 [30, 1021, 15, 23, 1],
 [30, 1021, 15, 23, 1, 275],
 [30, 1021, 15, 23, 1, 275, 4],
 [30, 1021, 15, 23, 1, 275, 4, 394],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21, 51],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21, 51, 1676],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21, 51, 1676, 2],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21, 51, 1676, 2, 18],
 [572, 51],
 [572, 51, 3398],
 [572, 51, 3398, 3399],
 [572, 51, 3398, 3399, 13],
 [572, 51, 3398, 3399, 13, 75],
 [572, 51, 3398, 3399, 13, 75, 817],
 [572, 51, 3398, 33

In [12]:
max_len = max([len(x) for x in input_sequences])

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding = 'pre')

In [14]:
padded_input_sequences

array([[   0,    0,    0, ...,    0,  145, 4790],
       [   0,    0,    0, ...,  145, 4790,    1],
       [   0,    0,    0, ..., 4790,    1, 1020],
       ...,
       [   0,    0,    0, ...,    3,  360,   83],
       [   0,    0,    0, ...,  360,   83,  358],
       [   0,    0,    0, ...,   83,  358, 1673]], dtype=int32)

In [15]:
x = padded_input_sequences[:, :-1]
x

array([[   0,    0,    0, ...,    0,    0,  145],
       [   0,    0,    0, ...,    0,  145, 4790],
       [   0,    0,    0, ...,  145, 4790,    1],
       ...,
       [   0,    0,    0, ..., 8931,    3,  360],
       [   0,    0,    0, ...,    3,  360,   83],
       [   0,    0,    0, ...,  360,   83,  358]], dtype=int32)

In [16]:
y = padded_input_sequences[:, -1]
y

array([4790,    1, 1020, ...,   83,  358, 1673], dtype=int32)

In [17]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y)
y.shape

(101619, 8932)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [19]:
model = Sequential()
model.add(Embedding(input_dim=tot_unique_word +1, output_dim=100, input_length=max_len))  # Explicit input shape
model.add(LSTM(150, return_sequences=True))  # Keep return_sequences=True for stacking LSTMs
model.add(LSTM(150))  # Second LSTM
model.add(Dense(tot_unique_word+1, activation='softmax'))  # Output layer

model.build(input_shape=(None, max_len))  # Manually build the model if necessary
model.summary()  # Check if parameters are built

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 20, 100)             │         893,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 20, 150)             │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 150)                 │         180,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 8932)                │       1,348,732 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,573,132 (9.82 MB)

 Trainable params: 2,573,132 (9.82 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [21]:
model.fit(x,y, epochs = 100)

Epoch 1/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 41s 10ms/step - accuracy: 0.0567 - loss: 6.6300
Epoch 2/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 29s 9ms/step - accuracy: 0.0989 - loss: 5.7378
Epoch 3/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 30s 9ms/step - accuracy: 0.1237 - loss: 5.3986
Epoch 4/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.1381 - loss: 5.1246
Epoch 5/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 40s 9ms/step - accuracy: 0.1509 - loss: 4.9120
Epoch 6/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.1618 - loss: 4.7237
Epoch 7/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 40s 9ms/step - accuracy: 0.1692 - loss: 4.5460
Epoch 8/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 30s 9ms/step - accuracy: 0.1757 - loss: 4.3774
Epoch 9/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.1869 - loss: 4.2113
Epoch 10/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.1971 - loss: 4.0553
Epoch 11/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 40s 9ms/step - accuracy: 0.2067 - loss: 3.9238
Epoch 1

In [1]:
#testing
import numpy as np
import time
text = "project"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=56, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)